# Imports

In [1]:
from IPython.display import Image
from IPython.core.display import HTML

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Configs

In [3]:
IMG_URL = 'https://i.ibb.co/47bRcVy/bow-normalized.jpg'

# Ex. 1

In [4]:
Image(url=IMG_URL,
     width=500, height=500)

In [5]:
texts = ['я и ты', 'ты и я', 'я я и только я', 'только не я', 'он'] # сразу удалил для удобства мешающие знаки

In [6]:
vect_count = CountVectorizer(token_pattern=r'(?u)\b\w+\b') # меняем патерн выделения токенов по-умолчанию

In [7]:
texts_df = vect_count.fit_transform(texts)

In [8]:
df = pd.DataFrame(texts_df.toarray(), columns=vect_count.get_feature_names(), index=texts)

In [9]:
df

,и,не,он,только,ты,я
я и ты,1,0,0,0,1,1
ты и я,1,0,0,0,1,1
я я и только я,1,0,0,1,0,3
только не я,0,1,0,1,0,1
он,0,0,1,0,0,0


In [10]:
number_of_documents = df.shape[0] # общее кол-во документов

In [11]:
# считаем кол-во вхождений признака
idf_count = {}
for col in df.columns:
    idf_count[col] = np.sum(df[col] >= 1) # кол-во схождений признака в корпус

In [12]:
idf_count

{'и': 3, 'не': 1, 'он': 1, 'только': 2, 'ты': 2, 'я': 4}

In [13]:
for k in idf_count.keys():
    idf_count[k] = np.log((1+number_of_documents)/(1+idf_count[k])) + 1

In [14]:
idf_count

{'и': 1.4054651081081644,
 'не': 2.09861228866811,
 'он': 2.09861228866811,
 'только': 1.6931471805599454,
 'ты': 1.6931471805599454,
 'я': 1.1823215567939547}

In [15]:
df = df.apply(lambda x: x / x.sum(), axis=1)

In [16]:
df

,и,не,он,только,ты,я
я и ты,0.333333,0.000000,0.0,0.000000,0.333333,0.333333
ты и я,0.333333,0.000000,0.0,0.000000,0.333333,0.333333
я я и только я,0.200000,0.000000,0.0,0.200000,0.000000,0.600000
только не я,0.000000,0.333333,0.0,0.333333,0.000000,0.333333
он,0.000000,0.000000,1.0,0.000000,0.000000,0.000000


In [17]:
for col in df.columns:
    df[col] = df[col] * idf_count[col]

In [18]:
df

,и,не,он,только,ты,я
я и ты,0.468488,0.000000,0.000000,0.000000,0.564382,0.394107
ты и я,0.468488,0.000000,0.000000,0.000000,0.564382,0.394107
я я и только я,0.281093,0.000000,0.000000,0.338629,0.000000,0.709393
только не я,0.000000,0.699537,0.000000,0.564382,0.000000,0.394107
он,0.000000,0.000000,2.098612,0.000000,0.000000,0.000000


# Ex. 2

In [19]:
df = pd.read_csv('labeled.csv')

In [20]:
df.head(2)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0


In [21]:
target = df['toxic']
del df['toxic']

In [22]:
target[:4]

0    1.0
1    1.0
2    1.0
3    1.0
Name: toxic, dtype: float64

In [23]:
df.head(2)

,comment
0,"Верблюдов-то за что? Дебилы, бл...\n"
1,"Хохлы, это отдушина затюканого россиянина, мол..."


In [24]:
# векторизуем тексты
vect_idf = TfidfVectorizer()
texts = vect_idf.fit_transform(df['comment']) 

In [25]:
cosine_similarity(texts[3], texts[12666])

array([[0.27330886]])

In [26]:
# не нолевое сходство
np.sort(cosine_distances(texts[43], texts))[0, 1:4]

array([0.83956018, 0.88210851, 0.88590919])

In [27]:
most_similary_texts = cosine_distances(texts[43], texts).argsort()[0,1:4]

In [28]:
for i, text in enumerate(df.loc[most_similary_texts]['comment']):
    print(f'Индекс текста: {most_similary_texts[i]}')
    print('Текст:')
    print(text)
    print()

Индекс текста: 1986
Текст:
НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?


Индекс текста: 1957
Текст:
Че за бригада и че за махоун? Из полицейской академии?

Индекс текста: 6259
Текст:
Герка ебет только даунов которые игрли а него. Ибо и геймплей и сюжетто кусок говна.




In [29]:
# Оригинальный текст
df.loc[43]

comment    Люди зажрались и охуели если по мнению этих иг...
Name: 43, dtype: object

# Ex. 3 

In [30]:
text_for_predict = open('data/2ch_corpus.txt').readlines()

In [31]:
text_for_predict[:2]

[" Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество 

In [32]:
import string
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [33]:
def preprocessing(text):
    text = text.lower()
    
    remove = string.punctuation
    remove += '«»—…#№“”'
    
    text = ' '.join([ morph.parse((word.strip(remove)))[0].normal_form for word in text.split(' ')])
    
    return text

## 1 подход

In [34]:
df['comment'] = df['comment'].apply(preprocessing)

In [35]:
df.head(4)

,comment
0,верблюд-то за что дебил бл...\n
1,хохол это отдушина затюканый россиянин мол вон...
2,собака собачий смерть\n
3,страница обновить дебил это тоже не оскорблени...


In [36]:
# обучаемся и тестируемся на текстах из labeled.csv, предсказываем для 2ch_corpus.txt

In [37]:
vect_tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df = 5,
    max_df = 0.9,
    max_features = 2000,
    token_pattern=r'(?u)\b\w+\b'
)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, test_size=0.2, stratify=target)

In [39]:
X_train = vect_tfidf.fit_transform(X_train)
X_test = vect_tfidf.transform(X_test)

In [40]:
clf_logreg = LogisticRegression(penalty='l2', n_jobs=-1, verbose=1, max_iter=1000)

In [41]:
clf_logreg.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished


LogisticRegression(max_iter=1000, n_jobs=-1, verbose=1)

In [42]:
y_pred = clf_logreg.predict(X_test)

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.94      0.89      1918
         1.0       0.85      0.63      0.72       965

    accuracy                           0.84      2883
   macro avg       0.84      0.79      0.81      2883
weighted avg       0.84      0.84      0.83      2883



In [44]:
# вроде норм качество

In [45]:
text_for_predict = [preprocessing(text) for text in text_for_predict]

In [46]:
# predict 
df_predict = vect_tfidf.transform(text_for_predict)

In [47]:
y_pred_proba = clf_logreg.predict_proba(df_predict)

In [48]:
max_10_index_logreg = (-y_pred_proba[:, 1]).argsort()[:10]

In [49]:
max_10_index_logreg

array([66922, 46397, 58455, 69075, 66927, 10868, 64544,  8084, 82566,
       33918])

In [50]:
max_10_prob_logreg = -np.sort((-y_pred_proba[:, 1]))[:10]

# 2 подход

In [51]:
vect_count = CountVectorizer(
    ngram_range=(1,2),
    min_df = 5,
    max_df = 0.8,
    max_features = 3000,
    token_pattern=r'(?u)\b\w+\b'
)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, test_size=0.2, stratify=target)

In [53]:
X_train = vect_count.fit_transform(X_train)
X_test = vect_count.transform(X_test)

In [72]:
clf_knn = KNeighborsClassifier(n_neighbors=10, n_jobs=-1, metric='cosine', algorithm='brute')

In [73]:
clf_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_jobs=-1,
                     n_neighbors=10)

In [74]:
y_pred = clf_knn.predict(X_test)

In [75]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.77      0.78      1918
         1.0       0.56      0.58      0.57       965

    accuracy                           0.71      2883
   macro avg       0.67      0.67      0.67      2883
weighted avg       0.71      0.71      0.71      2883



In [76]:
df_predict = vect_count.transform(text_for_predict)

In [77]:
y_pred_proba = clf_knn.predict_proba(df_predict)

In [78]:
max_10_index_knn = (-y_pred_proba[:, 1]).argsort()[:10]

In [79]:
max_10_prob_knn = -np.sort((-y_pred_proba[:, 1]))[:10]

# Сравниваем

## logreg

In [80]:
for i in range(len(max_10_index_logreg)):
    print(f'вероятность токсичности {max_10_prob_logreg[i]}')
    print('Текст')
    print(text_for_predict[max_10_index_logreg[i]])

вероятность токсичности 0.9996496786063355
Текст
 уиииииииииииия ряя ты тупой ряять уиииия не согласный с пидорашкой-ты хохол уиииииииии!стеклома накатить свинья руснявая.

вероятность токсичности 0.9995371523312854
Текст
 поздравлять ты хохол

вероятность токсичности 0.9993012735434366
Текст
 игор ты сука ты чмо ты пидор ты гной блядь ты нахуй варга убить сука как я теперь новый альбом ждать чмо всратое?

вероятность токсичности 0.9992913741890773
Текст
 ахи блядь андрей ты долбоести нахуй ты кроссовок фоткаешь)

вероятность токсичности 0.9992823736446385
Текст
  уиииия ряять не согласный с пидорашкой-ты хохол уиииииииии!пидорашка ты совсем тупой?+15 рублей

вероятность токсичности 0.999218527728381
Текст
 хохол ты обосраться он твой соотечественник.

вероятность токсичности 0.9991096292495616
Текст
 хохол радоваться евровидение хачёвка же победить ты чего!

вероятность токсичности 0.998975088737975
Текст
 ты запятая пропустилить ты просто тупой

вероятность токсичности 0.998892049948

## KNN

In [81]:
for i in range(len(max_10_index_knn)):
    print(f'вероятность токсичности {max_10_prob_knn[i]}')
    print('Текст')
    print(text_for_predict[max_10_index_knn[i]])

вероятность токсичности 1.0
Текст
 братец-маляр а кинуть я оригинальныйsongs asset из папка musicспасибо

вероятность токсичности 1.0
Текст
 на вторжение народ шугаться лужа от распространитель скверна лал.

вероятность токсичности 1.0
Текст
 маг блядь он портал кастать куда надо.

вероятность токсичности 1.0
Текст
 моторика рука поиск человек по интерес развитие реакции.

вероятность токсичности 1.0
Текст
   в твой жизнь прийдеть тян и всё перевернуть вный она быть тем ангел который скасит весь твой жизнь ты счастие светить ниу кто ещё такой карта невыпало.

вероятность токсичности 1.0
Текст
 хохол против хищника

вероятность токсичности 1.0
Текст
 но потом дух отвернуться от он 

вероятность токсичности 1.0
Текст
 ещё обидеться порватка я бы ты вообще посоветовать не лезть сюда а прочитать вот этот книга читаешь2 пилить параллельно свой говнопроектика приходить сюда точно знать что ты нужно плюс данный подхода1 вероятность что ты она прочитать и освоить 80%2 вероятность что замутить 

Тексты не совпаают, но все очень токсчиное. Вероятность 1 в версии с KNN т.к. он не умеет в вероятность.